<a href="https://colab.research.google.com/github/TSffer/Topicos-IA/blob/master/MLP_MVS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm, datasets

In [ ]:
def Leer_Datos(filename): 
    data = pd.read_csv(filename, sep=',', header=None).to_numpy()
    return data

def Normalizar_Datos(data):
    meam_data = data.mean(axis=0)
    std_data = data.std(axis=0)
    normalization_t = data - meam_data
    normalization_t = normalization_t / std_data
    return normalization_t

def Separar_X_y(data):
    n = data.shape[1]
    X = data[:, :n-1]
    y = data[:, n-1:]
    return X, y

def accuracy(y_, prediction):
    y_true = []
    y_pred = []

    for i in range(len(y_)):
        y_pred.append(np.argmax(prediction[i]))
        y_true.append(np.argmax(y_[i]))

    num_correct = 0
    for i in range(len(y_)):
        if(y_pred[i] == y_true[i]):
            num_correct += 1
    return num_correct /len(y_)   

def k_fols(data, k):
    data_n = Normalizar_Datos(data[:,0:data.shape[1]-1])
    print('Sub_matriz: ',data_n.shape)
    y =  data[:,-1]    
    print('Sin normalizar: ',data.shape)
    
    data = np.column_stack((data_n, y))

    print('Con normalizar: ', data.shape)
    clase_0 = np.count_nonzero( y == 0)
    clase_1 = np.count_nonzero( y == 1)
    clase_2 = np.count_nonzero( y == 2)

    div0 = int(clase_0/k)
    div1 = int(clase_1/k)
    div2 = int(clase_2/k)
    print('clase 0: ', div0)
    print('clase 1: ', div1)
    print('clase 2: ', div2)

    class_0 = data[0:clase_0, :]
    class_1 = data[clase_0:clase_0+clase_1, :]
    class_2 = data[clase_0+clase_1:clase_0+clase_1+clase_2, :]

    np.random.shuffle(class_0)
    np.random.shuffle(class_1)
    np.random.shuffle(class_2)
    
    size_fold = int(data.shape[0] / k)
    remainder_size_fold = int(data.shape[0] % k)
    data = data[:data.shape[0] - remainder_size_fold, :]

    k_fols = []
    idx_row0 = 0
    idx_row1 = 0
    idx_row2 = 0
    for i in range(k):
        X0 = class_0[idx_row0:idx_row0+div0, :]
        X1 = class_1[idx_row1:idx_row1+div1, :]
        X2 = class_2[idx_row2:idx_row2+div2, :]
        X = np.concatenate((X0, X1, X2))
        k_fols.append(X)
        idx_row0 += div0
        idx_row1 += div1
        idx_row2 += div2
    return k_fols, size_fold

In [ ]:
def Crear_Entrenamiento_Prueba(k):
    data = datasets.load_iris()
    x = data["data"]
    y = data["target"]
    y = y.reshape((150, 1))
    data = np.column_stack((x, y))
    k_f, siz = k_fols(data, k)

    for i in range(k):
        np.savetxt('/content/drive/My Drive/MVS_PM/fold_'+str(i)+'.csv', k_f[i], delimiter=',')
#Crear_Entrenamiento_Prueba(3)

In [ ]:
def Sigmoid_(z):
    return 1 / (1 + np.exp(-z))

def dS(z):
    return Sigmoid_(z) * (1 - Sigmoid_(z))
    
def delta(y_true, y_pred):
    return (y_pred - y_true) * dS(y_pred)

def Crear_w(dimensions):
    n_layers = len(dimensions)
    w = {}
    b = {}

    #print(n_layers)
    for i in range(len(dimensions) - 1):
        w[i + 1] = np.random.randn(dimensions[i], dimensions[i + 1]) / np.sqrt(dimensions[i])
        b[i + 1] = np.zeros(dimensions[i + 1])

    return w, b, n_layers 
        

def Forward(x, w, b, n_layers):
    z = {}
    A = {1: x}

    for i in range(1, n_layers):
        z[i+1] = np.dot(A[i], w[i]) + b[i]
        A[i+1] = Sigmoid_(z[i+1])
        
    return z, A

def Backward(z, A, y, w, b, learning_rate, n_layers):
    delta_ = delta(y, A[n_layers])
    dw = np.dot(A[n_layers-1].T, delta_)

    update_params = {
        n_layers - 1: (dw, delta_)
    }

    for i in reversed(range(2, n_layers)):
        delta_ = np.dot(delta_, w[i].T) * dS(z[i])
        dw = np.dot(A[i-1].T, delta_)
        update_params[i-1] = (dw, delta_)
        
    for k, v in update_params.items():
        w[k] -= learning_rate * v[0]
        b[k] -= learning_rate * np.mean(v[1], 0) 
    
    return w, b
        
def predict(x, w, b, n_layers):
    _, a = Forward(x, w, b, n_layers)
    return a[n_layers]

def Costs(y_true, y_pred):
    return np.mean((y_pred - y_true)**2)

def Calcular_Funcion_Costo(A, y):
    result = np.sum((y * np.log(A)) + ((1 - y) * np.log(1-A)))
    result = (-result)/y.shape[0]
    #print("y.shape[1]", y.shape[1])
    return np.mean(result)

def Gradiente_Descendiente(x, y, w, b, n_layers,epochs, learning_rate = 1e-3):
    if not x.shape[0] == y.shape[0]:
        raise ValueError("Length of x and y arrays dont's match")
    
    cost_h = np.zeros(epochs)

    for i in range(epochs):
        z, a = Forward(x, w, b, n_layers)
        w, b = Backward(z, a, y, w, b, learning_rate, n_layers)
        #cost_h[i] = Calcular_Funcion_Costo(a[n_layers], y)
        cost_h[i] = Costs(y, a[n_layers])
        #    _, a = Forward(x, w, b, n_layers)
    return w, b, cost_h
                    

In [ ]:
# Experimento 1

def EXP1(filename, dataname, arq):
    k_fols = []
    
    for i in range(3):
        X = Leer_Datos(filename +'/fold_'+str(i)+'.csv')
        k_fols.append(X)
    
    for conf in arq:
        apr = np.array([0.01, 0.05, 0.1, 0.2, 0.3, 0.4])
        s = 0
        sum_acc = 0
        acc = np.zeros((7,6), float)

        w, b, n_layers = Crear_w(conf)

        for it in range(500,4000,500):
            for i in range(len(apr)):
                for k in range(3):
                    data_train = np.concatenate((k_fols[(k+1)%3], k_fols[(k+2)%3]))
                    data_test = k_fols[k]
                    train_x, train_y = Separar_X_y(data_train)
                    train_y = train_y.reshape(train_y.shape[0])
                    train_y = np.eye(conf[len(conf)-1])[train_y.astype(int)]
                    test_x, test_y = Separar_X_y(data_test)
                    test_y = test_y.reshape(test_y.shape[0])
                    test_y = np.eye(conf[len(conf)-1])[test_y.astype(int)]
                    
                    w, b, cost_h = Gradiente_Descendiente(train_x, train_y, w, b, n_layers, 1000, learning_rate = 0.01)
                    prediction = predict(test_x, w, b, n_layers)
                    #nn.fit(train_x, train_y, loss = MSE, epochs=it, batch_size=150, learning_rate=apr[i])
                    #prediction = nn.predict(test_x)
                    sum_acc += accuracy(test_y, prediction)
                acc[s,i] = sum_acc/3
                sum_acc = 0.0
            s = s + 1  
        
        print("Conjunto de Datos: " + dataname + "       Aquitectura: ", conf)
        fils = ["iter: 500", "iter: 1000", "iter: 1500", "iter: 2000", "iter: 2500", "iter: 3000", "iter: 3500"]
        cols = ["α: 0.01", "α: 0.05", "α: 0.1", "α: 0.2", "α: 0.3", "α: 0.4"]
        df = pd.DataFrame(acc, columns=cols, index=fils)

        print(df)
        print('\n')

# Dataset Iris
arquitecture1 = [(4, 4, 3), (4, 1, 2, 3), (4, 1, 2, 3, 3)]
arquitecture2 = [(4, 2, 3), (4, 2, 3, 3), (4, 4, 5, 4, 3)]

# Dataset Enfermedad Cardiaca
arquitecture3 = [(13, 3, 2), (13, 3, 3, 2), (13, 5, 7, 3, 2)]
arquitecture4 = [(13, 13, 2), (13, 13, 15, 2), (13, 4, 15, 4, 2)]

EXP1("/content/drive/My Drive/MVS_PM", "Iris", arquitecture1)
EXP1("/content/drive/My Drive/MVS_PM", "Iris", arquitecture2)

EXP1("/content/drive/My Drive/MVS_PM/Enfermedad_Cardiaca_Data", "Enfermedades Cardiaca", arquitecture3)
EXP1("/content/drive/My Drive/MVS_PM/Enfermedad_Cardiaca_Data", "Enfermedades Cardiaca", arquitecture4)

Conjunto de Datos: Iris       Aquitectura:  (4, 4, 3)
             α: 0.01   α: 0.05    α: 0.1    α: 0.2    α: 0.3    α: 0.4
iter: 500   0.923611  0.965278  0.965278  0.965278  0.965278  0.965278
iter: 1000  0.965278  0.965278  0.965278  0.958333  0.958333  0.958333
iter: 1500  0.958333  0.958333  0.958333  0.958333  0.958333  0.965278
iter: 2000  0.965278  0.965278  0.965278  0.965278  0.965278  0.965278
iter: 2500  0.965278  0.965278  0.965278  0.965278  0.965278  0.965278
iter: 3000  0.965278  0.965278  0.965278  0.965278  0.965278  0.972222
iter: 3500  0.979167  0.979167  0.979167  0.986111  0.986111  0.986111


Conjunto de Datos: Iris       Aquitectura:  (4, 1, 2, 3)
             α: 0.01   α: 0.05    α: 0.1    α: 0.2    α: 0.3    α: 0.4
iter: 500   0.833333  0.958333  0.965278  0.965278  0.965278  0.965278
iter: 1000  0.958333  0.958333  0.958333  0.965278  0.965278  0.972222
iter: 1500  0.972222  0.972222  0.972222  0.972222  0.979167  0.979167
iter: 2000  0.979167  0.979167  0.9

In [ ]:
# Experimento 2

def EXP2(filename, dataname):
    kernels = ['linear', 'poly', 'rbf']
    C = np.array([2500.0, 1000.0, 100.0, 1.0, 0.5, 0.01])

    k_fols = []
    for i in range(3):
        X = Leer_Datos(filename +'/fold_'+str(i)+'.csv')
        k_fols.append(X)
    
    s = 0
    sum_acc = 0
    acc = np.zeros((6,3), float)

    for c in C:
        for i in range(len(kernels)):
            for k in range(3):
                data_train = np.concatenate((k_fols[(k+1)%3], k_fols[(k+2)%3]))
                data_test = k_fols[k]
                train_x, train_y = Separar_X_y(data_train)          
                test_x, test_y = Separar_X_y(data_test)
                model = svm.SVC(C=c, kernel=kernels[i], gamma='scale')
                model.fit(train_x, train_y.reshape(train_y.shape[0]))
                acc_test = model.score(test_x, test_y.reshape(test_y.shape[0]))
                
                
                sum_acc += acc_test
            
            acc[s,i] = sum_acc/3
            sum_acc = 0.0
        s = s + 1  

    print("Conjunto de Datos: " + dataname)
    fils = ["C: 2500.0", "C: 1000.0", "C: 100.0", "C: 1.0", "C: 0.5", "C: 0.01"]
    cols = ["Kernel: linear", "Kernel: poly", "Kernel: rbf"]
    df = pd.DataFrame(acc, columns=cols, index=fils)

    print(df)
    print('\n')

EXP2("/content/drive/My Drive/MVS_PM", "Iris")
EXP2("/content/drive/My Drive/MVS_PM/Enfermedad_Cardiaca_Data", "Enfermedad Cardiaca")

Conjunto de Datos: Iris
           Kernel: linear  Kernel: poly  Kernel: rbf
C: 2500.0        0.986111      0.916667     0.958333
C: 1000.0        0.979167      0.916667     0.958333
C: 100.0         0.979167      0.958333     0.944444
C: 1.0           0.986111      0.923611     0.965278
C: 0.5           0.972222      0.888889     0.958333
C: 0.01          0.868056      0.527778     0.909722


Conjunto de Datos: Enfermedad Cardiaca
           Kernel: linear  Kernel: poly  Kernel: rbf
C: 2500.0        0.838284      0.745875     0.801980
C: 1000.0        0.838284      0.745875     0.801980
C: 100.0         0.838284      0.745875     0.801980
C: 1.0           0.834983      0.798680     0.844884
C: 0.5           0.838284      0.772277     0.841584
C: 0.01          0.838284      0.544554     0.544554




In [ ]:
def Plot(filename, dataname, epoch, apr, model):
    k_fols = []
    for i in range(3):
        X = Leer_Datos(filename +'/fold_'+str(i)+'.csv')
        k_fols.append(X)

    cl = ['b:','r:','y:']
    cost_ht = []
    w, b, n_layers = Crear_w(model)
    sum_acc = 0
  
    data_train = np.concatenate((k_fols[0], k_fols[1], k_fols[1]))
        
    train_x, train_y = Separar_X_y(data_train)
    train_y = train_y.reshape(train_y.shape[0])
    train_y = np.eye(3)[train_y.astype(int)]

    w, b, cost_h = Gradiente_Descendiente(train_x, train_y, w, b, n_layers, epoch, learning_rate = apr)

    cost_ht.append(cost_h)
        
    plt.plot(np.arange(cost_ht[0].shape[0]),cost_ht[0], cl[0], label='Iris')
    plt.title(dataname)
    plt.xlabel('Iteraciones')
    plt.ylabel('Costo')
    plt.legend()
    plt.show()


Plot("/content/drive/My Drive/MVS_PM", "Iris", 3500, 0.3, (4, 3, 3))